In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import graph_tool.all as gt

import json
import random
import nmi
import glob
import string

from pathlib import Path
from matplotlib.ticker import FormatStrFormatter
from time import localtime, strftime
from sbmtm import sbmtm
from nsbm import nsbm

from helps import *

import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn')
plt.rcParams['font.size'] = 15
plt.rcParams["xtick.labelsize"]=13
plt.rcParams["ytick.labelsize"]=13
plt.rcParams["axes.titlesize"]=15
plt.rcParams["figure.dpi"]=600
plt.rcParams["savefig.format"]="pdf"
plt.rcParams["savefig.bbox"]="tight"

# hSBM: experiment run

Here we show how to reproduce all the experiments with hSBM. To select the desidered experiment change the value of variable dataset choosing one of the three following values: hSBM-mRNA, hSBM-lncRNA, hSBM-mRNA-lncRNA.

In [ ]:
dataset="hSBM-mRNA"
print(dataset)
df=pd.read_csv(f"Results/{dataset}/{dataset}.csv.gz",index_col=0)
df=df.sample(frac=1, axis=1)
df.head(3)

In [ ]:
model=sbmtm()
model.make_graph_from_BoW_df(df)
V=model.get_V()
D=model.get_D()
print(V/(D*(D-1)))

In [ ]:
# 3 minutes for the hSBM-mRNA experiment
# 1 minute for the hSBM-lncRNA experiment
# 3 minutes for the hSBM-mRNA-lncRNA experiment
#with a i5-8265U 4 cores 1.60 GHz laptop

print(strftime("%Y-%m-%d %H:%M:%S", localtime()))
model.fit(n_init=1)
print(strftime("%Y-%m-%d %H:%M:%S", localtime()))

In [ ]:
Path(f"Results/{dataset}/hSBM").mkdir(parents=True, exist_ok=True)
model.save_graph(filename=f"Results/{dataset}/hSBM/{dataset}-graph.xml.gz")
model.save_levels(f"Results/{dataset}/hSBM/{dataset}")

# hSBM: experiment analysis
Here you can find all the steps we followed to analyse the outcome of all the hSBM's experiments.
You can reproduce all the experiments changing the variable "dataset" that corresponds to the experiment's name.

In [ ]:
dataset="hSBM-mRNA"
print(dataset)
info=pd.read_csv("HelperFiles/ENS-Info.txt",index_col=0,sep="\t")
df=pd.read_csv(f"Results/{dataset}/{dataset}.csv.gz",index_col=0)
df.shape

In [ ]:
labels=pd.read_csv("HelperFiles/All-datasets-labels.csv",index_col=0)
labels=labels.loc[df.columns]
labels.index=labels.index.astype(str)
subtypes=list(sorted(set(labels.typehisto)))
print(df.shape)
df.head()

In [ ]:
performances={}
for level in [0,1,2]:
    with open(f"Results/{dataset}/hSBM/{dataset}-cluster-level-{level}.txt") as f:
        clusters=json.load(f)
    cluster_df=pd.DataFrame.from_dict(clusters,orient="index")
    labels["hSBM"]="--"
    for i in range(len(clusters)):
        labels["hSBM"].loc[np.asarray(np.asarray(clusters[str(i)])[:,0])]=i
    
    NMI=np.around(nmi.compute_normalised_mutual_information(labels.typehisto,labels["hSBM"]),decimals=3)
    nmi_rand=0
    for k in range(1000):
        a=labels["hSBM"].to_list()
        np.random.shuffle(a)
        nmi_rand+=nmi.compute_normalised_mutual_information(labels["typehisto"],a)/1000

    performances[f"Level {level}"]=[NMI,NMI/nmi_rand]
with open(f"Results/{dataset}/hSBM/{dataset}_NMI.json", 'w') as fp:
    json.dump(performances, fp)

In [ ]:
with open(f"Results/{dataset}/hSBM/{dataset}_NMI.json") as f:
        performances=json.load(f)
for key in performances.keys():
    print(f"{key} NMI: {performances[key][0]}, NMI/NMI*: {int(performances[key][1])}")

# NMI

In [ ]:
with open(f"Results/{dataset}/hSBM/{dataset}_NMI.json") as f:
        perfs=json.load(f)
perfs

In [ ]:
fig, axs = plt.subplots(1,3,figsize=(25,8), dpi=600)
axs=axs.flatten()
ax=0
for level in [0,1,2]:
    with open(f"Results/{dataset}/hSBM/{dataset}-cluster-level-{level}.txt") as f:
        clusters=json.load(f)
    cluster_df=pd.DataFrame.from_dict(clusters,orient="index")
    labels["hSBM"]="--"
    for i in range(len(clusters)):
        labels["hSBM"].loc[np.asarray(np.asarray(clusters[str(i)])[:,0])]=i
     
    labels["typehisto_1"]=pd.Series(list(labels["typehisto"])).astype('category').cat.codes.values    
    fraction_sites = pd.DataFrame(index=labels["hSBM"].unique(), columns=sorted(labels["typehisto_1"].unique())[::-1]).fillna(0)
    for sample in labels[["hSBM","typehisto_1"]].values:
        fraction_sites.at[sample[0],sample[1]] += 1

    fraction_sites = fraction_sites.sort_values(by=list(fraction_sites.columns), ascending=True)
    fraction_sites.columns=subtypes[::-1]
    fraction_sites.index=[i for i in range(len(fraction_sites))]
    fraction_sites.plot.bar(stacked=True, color=dict(zip(subtypes, nmi.set_colors(subtypes))),
                           width=1, alpha=0.65, ax=axs[level])    
       
    axs[level].set_xlabel(f"\nClustering level {level}", size=25, weight='bold')
    axs[level].set_ylabel("Number of cells", size=25, weight='bold')
    axs[level].yaxis.set_major_formatter(FormatStrFormatter('%.0f'))
    axs[level].tick_params(axis='both', which='major', labelsize=25, rotation=0)
    
    legend_properties = {'weight':'bold', "size":"x-large"}
    if level==1:
        axs[level].legend(loc=(0.55,0.65), prop=legend_properties)
    else:
        axs[level].get_legend().remove()        
    
    axs[level].text(-0.055, 1.1, string.ascii_uppercase[level],
                 transform=axs[ax].transAxes, size=35, weight='bold',rotation=0)
    axs[level].xaxis.set_major_locator(plt.MaxNLocator(min(12, len(set(labels.hSBM))+1)))
    ax+=1   
    
fig.tight_layout(pad=1)    
title=f"Fig_1"
plt.savefig(f"Results/Figures/{title}.png", dpi=600)
plt.savefig(f"Results/Figures/{title}.pdf", dpi=600)
plt.show()

# Data reading

In [ ]:
level=1
data=hSBM_data(dataset=dataset,labels=labels, lev=level, info=info)
print(data.keys())

labels=data["sample_cluster"]
topic_gene_prob=data["topic_gene_prob"]
topic_gene=data["topic_gene"]
topic_gene_genename=data["topic_gene_genename"]
topic_gene_raw=data["topic_gene_raw"]
p_topic_sample_cent=data["p_topic_sample_cent"]
p_topic_sample=data["p_topic_sample"]


Path(f"Results/{dataset}/hSBM/Data").mkdir(parents=True, exist_ok=True)
for key in data.keys():
    data[key].to_csv(f"Results/{dataset}/hSBM/Data/{key}-level-{level}.csv")


print("N clusters: ", len(set(labels.hSBM)))
print("N topics:", p_topic_sample_cent.shape[1])

clusters=sorted(list(set(labels.hSBM)))

p_topic_class_cent=pd.DataFrame(index=clusters, columns=p_topic_sample.columns)    
for cla in clusters:
    in_sample=labels[labels["hSBM"]==cla].index
    p_topic_class_cent.loc[cla]=p_topic_sample_cent.loc[in_sample].mean() 
p_topic_class_cent.index.names=["clusters"]
data["p_topic_class_cent"]=p_topic_class_cent

path_to_save=f"Results/{dataset}/hSBM/Outcome analysis/Level {level}/"
Path(f"{path_to_save}").mkdir(parents=True, exist_ok=True)

# Topic - cluster assignment
Here we assign to each cluster the most appropriate topics following the method explained in the paper.

In [ ]:
topic_arr, threshold = loop_topics(clusters, p_topic_class_cent, direction="up")
with open(f"{path_to_save}/{dataset} hSBM level {level} Topic up clusters threshold {np.around(threshold, decimals=2)}.json", 'w') as fp:
    json.dump(topic_arr, fp)
topic_arr

In [ ]:
assigned_topic=flat_list([topic_arr[k] for k in topic_arr.keys()])
assigned_topic

## Enrichement test

In [ ]:
#Here you need to choose the appropriate file between "MSigDB.json" for the hSBM-mRNA experiment 
#and "lncSEA_red.json" for the hSBM-lncRNA experiment. 

path=f"Results/{dataset}/hSBM/Outcome analysis/Level {level}"
Path(f"{path}/Enrichment test topics").mkdir(parents=True, exist_ok=True)
with open('HelperFiles/MSigDB.json') as f:
    all_lists=json.load(f)
print(len(all_lists.keys()))

In [ ]:
#Please note that this is the slowest step of the procedure: it takes about 20-30 seconds to test 
#each topic with a i5-8265U 4 cores 1.60 GHZ laptop
enrichment_test(all_lists, topic_gene_genename, topic_gene_raw, info, dataset[5:],
               f"{path}/Enrichment test topics", dataset, 1)

In [ ]:
path=f"Results/{dataset}/hSBM/Outcome analysis/Level {level}"
dfs_hgt={}
for topic in topic_gene.columns:
    dfs_hgt[topic]=pd.read_csv(f"{path}/Enrichment test topics/{dataset} level {level} Enrichment Test {dataset[5:]}-topic {topic}.csv",index_col=0)
    dfs_hgt[topic].drop(dfs_hgt[topic][dfs_hgt[topic].fdr<3].dropna().index,inplace=True)
    dfs_hgt[topic].sort_values(by="fdr",inplace=True,ascending=False)

Before calling the function topics_names check if in the first 10-20 positions of each dataframe in the dictionary dfs_hgt there are gene sets that totally don't matter with your data.
After that topics_names has run, you should check the assigment makes sense, otherwise you can manually change it. 

Please not that the developed procedure doesn't take in account the biological meaning of gene sets in the database, it's just a fully reproducibile method that assign the best "name" to each topic.

In [ ]:
topic_name=topics_names(topics=assigned_topic,enr_test_outcome=dfs_hgt,
                        database=all_lists)
topic_name.sort_index()